In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
# Training settings
batch_size = 64

In [0]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True, # represents the training section of the data
                               transform=transforms.ToTensor(),
                               download=True)

In [0]:
test_dataset = datasets.MNIST(root='./data/',
                              train=False, #validation set 
                              transform=transforms.ToTensor())


In [0]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [0]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [0]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        '''
        nn.Conv2d(in_channels, out_channels, kernel_size)
        in_channels - number of channels of the input
                    - for greyscale images we have 1, or basically a single channel
                    - for RGB images we have 3 channels and for sattelite date number can be higher.
        
        out_channels - number of different filters applied to the input images and the resulting images from the operation
                     - in our case is 10 at first conv, so we will have 10 different images as a result of filters applied
        '''
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2) # apply max pool with a window of 2X2
        self.fc = nn.Linear(320, 10) # final fully-connected layer

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return x

In [0]:
model = Net()

In [0]:
print(model)

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [0]:
train_loss = []
validation_loss=[]
validation_accuracy=[]
train_accuracy=[]

In [0]:
def train(epoch):
    model.train()
    correct = 0
    losses=[]
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_loss.append(sum(losses)/len(losses))
    train_accuracy.append(correct* 100./len(train_loader.dataset))

In [0]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    losses=[]
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            # sum up batch loss
            loss= criterion(output, target)
            losses.append(loss.item())
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss = sum(losses)/len(losses)
        validation_loss.append(test_loss)
        validation_accuracy.append(100. * correct/len(test_loader.dataset))
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))


In [0]:
for epoch in range(1, 10):
    train(epoch)
    test()

In [0]:
#matplotlib este folosit pentru a realiza grafice
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [0]:
# Printăm comparativ cu roșu loss-ul de pe setul de validare și cu albastru loss-ul de pe setul de validare.
plt.plot(train_loss, "r-", validation_loss, "b-")

In [0]:
plt.plot(train_accuracy, "r-", validation_accuracy, "b-")